In [3]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from random import randint
from time import sleep, strftime
import smtplib
from datetime import datetime

In [4]:
driver = webdriver.Chrome()
sleep(2)

In [5]:
kayak = "https://www.kayak.com/flights/CMH-ROM/2023-06-29/2023-07-04?sort=bestflight_a"
driver.get(kayak)

In [6]:
# Load more results to maximize the scraping

def load_more():
    #try:
    more_results = '//a[@class = "moreButton"]'
    results = driver.find_element(By.XPATH, more_results).click()
    print('sleeping...')
    #sleep(randint(1,5))
    #except

In [7]:
search_modes = {'best':'bestflight', 'cheap':'price', 'quick':'duration'}
def change_mode_with_button(mode):
        print('Changing to ' + mode + 'mode.')
        mode_button_xpath = f'//a[@data-code = "{search_modes[mode]}"]'
        driver.find_elements(By.XPATH, mode_button_xpath)[0].click()
        print('sleeping...')
        #sleep(randint(3,10))

In [8]:
all_field_xpath = {#'Results':'//*[@class = "resultWrapper"]',
                   'Depart_Time':'//*[@class = "section times"]//*[@class = "time-pair"][1]',  #'//*[@class = "depart-time base-time"]',
                   'Arrival_Time':'//*[@class = "section times"]//*[@class = "time-pair"][2]',  #'//*[@class = "arrival-time base-time"]',
                   'Stops':'//*[@class = "stops-text"]',
                   'Duration':'//*[@class = "section duration allow-multi-modal-icons"]/*[@class = "top"]',
                   'Cities':'//*[@class = "section duration allow-multi-modal-icons"]/*[@class = "bottom"]',
                   'Prices':'//*[@class="price option-text"]'}

In [9]:
def get_data(xpath):
    results_raw = driver.find_elements(By.XPATH, xpath)
    results_text = [" ".join(result.text.split('\n')) for result in results_raw if result.text != '']
    return results_text

In [10]:
def get_data_from_all_fields():
    departure_data = {}
    return_data = {}
    prices = []
    
    for key, val in all_field_xpath.items():
        results = get_data(val)
        
        # Price is the only field that's only one per result
        if key == 'Prices':
            prices = results
            continue
        
        departure_data[key] = results[::2]
        return_data[key] = results[1::2]
        
    return departure_data, return_data, prices

In [11]:
def display_dic(dic):
    for key, val in dic.items():
        print(f'{len(val)}, {key}: {val}')

In [12]:
def generate_flights_df(departure_data, return_data, prices):
    # Creating both data frames for departure and return and putting them together
    departure_df = pd.DataFrame.from_dict(departure_data)
    return_df = pd.DataFrame.from_dict(return_data)
    full_df = pd.concat([departure_df, return_df], axis=1)
    
    # Adding the multilavel column to the data frame
    column_array = [['Departure', 'Return'], departure_df.columns]
    column_index = pd.MultiIndex.from_product(column_array)
    full_df.columns = column_index
    
    # Adding the price column
    full_df['Price'] = prices
    
    return full_df

In [13]:
load_more()

sleeping...


In [14]:
departure_data, return_data, prices = get_data_from_all_fields()
display_dic(departure_data)
display_dic(return_data)

47, Depart_Time: ['7:23 pm', '7:23 pm', '7:23 pm', '7:23 pm', '7:23 pm', '2:11 pm', '2:45 pm', '2:11 pm', '7:23 pm', '12:12 pm', '7:23 pm', '5:49 pm', '5:49 pm', '12:12 pm', '2:11 pm', '7:23 pm', '7:23 pm', '12:12 pm', '7:23 pm', '9:59 am', '2:11 pm', '2:11 pm', '7:23 pm', '5:49 pm', '7:23 pm', '12:19 pm', '5:49 pm', '2:11 pm', '12:12 pm', '9:59 am', '5:49 pm', '12:19 pm', '7:23 pm', '2:11 pm', '7:23 pm', '7:23 pm', '7:23 pm', '4:07 pm', '7:23 pm', '7:23 pm', '4:25 pm', '7:23 pm', '7:23 pm', '12:12 pm', '2:45 pm', '9:59 am', '12:12 pm']
47, Arrival_Time: ['12:55 pm +1', '12:55 pm +1', '12:55 pm +1', '12:55 pm +1', '12:55 pm +1', '9:10 am +1', '8:15 am +1', '9:10 am +1', '12:55 pm +1', '7:45 am +1', '12:55 pm +1', '12:55 pm +1', '12:55 pm +1', '7:45 am +1', '9:10 am +1', '12:55 pm +1', '12:55 pm +1', '7:45 am +1', '12:55 pm +1', '7:15 am +1', '9:10 am +1', '9:10 am +1', '12:55 pm +1', '12:55 pm +1', '12:55 pm +1', '8:00 am +1', '12:55 pm +1', '9:10 am +1', '7:45 am +1', '7:15 am +1', '1

In [15]:
flights_df = generate_flights_df(departure_data, return_data, prices)
flights_df

Departure                                                Return  \
   Depart_Time Arrival_Time    Stops Duration     Cities Depart_Time   
0      7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO     5:15 pm   
1      7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    11:05 am   
2      7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO     7:45 pm   
3      7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    10:30 am   
4      7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    10:30 am   
5      2:11 pm   9:10 am +1   1 stop  12h 59m  CMH ‐ FCO    11:05 am   
6      2:45 pm   8:15 am +1   1 stop  11h 30m  CMH ‐ FCO    10:30 am   
7      2:11 pm   9:10 am +1   1 stop  12h 59m  CMH ‐ FCO    10:30 am   
8      7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    12:50 pm   
9     12:12 pm   7:45 am +1   1 stop  13h 33m  CMH ‐ FCO    11:05 am   
10     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    11:05 am   
11     5:49 pm  12:55 pm +1  2 stops  13h 06m  CMH ‐ FCO    11:05 am   
12     5:49 pm  12:55 pm +1  2 stops  13h 06m  CMH ‐ FCO    10:30 am   
13    12:12 pm   7:45 am +1   1 stop  13h 33m  CMH ‐ FCO    10:30 am   
14     2:11 pm   9:10 am +1   1 stop  12h 59m  CMH ‐ FCO    10:30 am   
15     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    12:50 pm   
16     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO     1:45 pm   
17    12:12 pm   7:45 am +1   1 stop  13h 33m  CMH ‐ FCO    10:30 am   
18     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO     6:20 am   
19     9:59 am   7:15 am +1   1 stop  15h 16m  CMH ‐ FCO    11:05 am   
20     2:11 pm   9:10 am +1   1 stop  12h 59m  CMH ‐ FCO    10:30 am   
21     2:11 pm   9:10 am +1   1 stop  12h 59m  CMH ‐ FCO    12:50 pm   
22     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    11:05 am   
23     5:49 pm  12:55 pm +1  2 stops  13h 06m  CMH ‐ FCO    11:05 am   
24     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    10:10 am   
25    12:19 pm   8:00 am +1   1 stop  13h 41m  CMH ‐ FCO    10:30 am   
26     5:49 pm  12:55 pm +1  2 stops  13h 06m  CMH ‐ FCO    11:05 am   
27     2:11 pm   9:10 am +1   1 stop  12h 59m  CMH ‐ FCO     1:45 pm   
28    12:12 pm   7:45 am +1   1 stop  13h 33m  CMH ‐ FCO    12:50 pm   
29     9:59 am   7:15 am +1   1 stop  15h 16m  CMH ‐ FCO    10:30 am   
30     5:49 pm  12:55 pm +1  2 stops  13h 06m  CMH ‐ FCO    10:30 am   
31    12:19 pm   8:00 am +1   1 stop  13h 41m  CMH ‐ FCO    11:05 am   
32     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    10:30 am   
33     2:11 pm   9:10 am +1   1 stop  12h 59m  CMH ‐ FCO    10:30 am   
34     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    11:10 am   
35     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    12:50 pm   
36     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO     1:45 pm   
37     4:07 pm   9:00 am +1   1 stop  10h 53m  CMH ‐ FCO    10:10 am   
38     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    11:05 am   
39     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO     8:05 am   
40     4:25 pm  11:15 am +1   1 stop  12h 50m  CMH ‐ FCO    11:35 am   
41     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    10:10 am   
42     7:23 pm  12:55 pm +1   1 stop  11h 32m  CMH ‐ FCO    10:30 am   
43    12:12 pm   7:45 am +1   1 stop  13h 33m  CMH ‐ FCO    11:05 am   
44     2:45 pm   8:15 am +1   1 stop  11h 30m  CMH ‐ FCO    10:30 am   
45     9:59 am   7:15 am +1   1 stop  15h 16m  CMH ‐ FCO    11:05 am   
46    12:12 pm   7:45 am +1   1 stop  13h 33m  CMH ‐ FCO    10:30 am   

                                               Price  
   Arrival_Time    Stops Duration     Cities          
0   10:24 pm +1   1 stop  35h 09m  FCO ‐ CMH  $1,486  
1       6:58 pm   1 stop  13h 53m  FCO ‐ CMH  $1,653  
2   10:24 pm +1  2 stops  32h 39m  FCO ‐ CMH  $1,474  
3       5:30 pm   1 stop  13h 00m  FCO ‐ CMH  $1,738  
4       6:49 pm   1 stop  14h 19m  FCO ‐ CMH  $1,738  
5       6:58 pm   1 stop  13h 53m  FCO ‐ CMH  $1,652  
6       6:40 pm   1 stop  14h 10m  F

In [30]:
class Flight_Scrapper:
    def __init__(self):
        # Creating a logger object to write progress
        self.logger = Logger()
        self.sleep_time_min = 5
        self.sleep_time_max = 11

        # Creating a web driver to browse the internet
        self.logger.log('Initiating Chrome driver instance')
        self.driver = webdriver.Chrome()
        self.current_url = ""

        # Parameters and info to do the flight search
        self.search_modes = {'best':'bestflight', 'cheap':'price', 'quick':'duration'}
        self.all_field_xpath = {'Wrappers':'//*[@class = "resultWrapper"]',
                                'Depart_Time':'//*[@class = "section times"]//*[@class = "time-pair"][1]',  #'//*[@class = "depart-time base-time"]',
                                'Arrival_Time':'//*[@class = "section times"]//*[@class = "time-pair"][2]',  #'//*[@class = "arrival-time base-time"]',
                                'Stops':'//*[@class = "section stops"]',
                                'Duration':'//*[@class = "section duration allow-multi-modal-icons"]/*[@class = "top"]',
                                'Cities':'//*[@class = "section duration allow-multi-modal-icons"]/*[@class = "bottom"]',
                                'Prices':'//*[@class="price option-text"]'}

        self.logger.log('Waiting to make sure driver starts properly, sleeping...')
        sleep(5)

    def load_more(self):
        self.logger.log('Attempting to load more results...')

        try:
            more_results_xpath = '//a[@class = "moreButton"]'
            self.driver.find_element(By.XPATH, more_results_xpath).click()

            self.logger.log('More results loaded, sleeping...')
            sleep(randint(self.sleep_time_min, self.sleep_time_max))
        except:
            self.logger.log('Something went wrong, could not get more results...')

    def change_mode_with_button(self,mode):
        self.logger.log('Changing to ' + mode + ' mode.')

        try:
            mode_button_xpath = f'//a[@data-code = "{self.search_modes[mode]}"]'
            self.driver.find_elements(By.XPATH, mode_button_xpath)[0].click()

            self.logger.log(f'{mode} mode active, sleeping...')
            sleep(randint(self.sleep_time_min, self.sleep_time_max))
        except Exception as e:
            print(e)
            self.logger.log('Something went wrong, could not change mode manually...')

    # Possible modes: cheap, best, quick
    def start_kayak(self, origin, destination, start_dates, end_dates, mode):
        # Check if user wants to check a mode in particular
        add_mode = ''
        if mode is not None:
            add_mode = '?sort={self.search_modes[mode]}_a'

        kayak = (f"https://www.kayak.com/flights/{origin}-{destination}/{start_dates}/{end_dates}{add_mode}")
        self.logger.log('Trying to get ' + kayak)

        try:
            # Getting the website
            self.driver.get(kayak)

            self.logger.log('Successfully accessed kayak, sleeping...')
            sleep(randint(self.sleep_time_min, self.sleep_time_max))
        except:
            self.logger.log(f'Something went wrong, could not get {kayak}...')

    def get_raw_data(self, xpath):
        results = self.driver.find_elements(By.XPATH, xpath)
        return [result.text for result in results]

    def get_data(self, xpath):
        results_raw = self.driver.find_elements(By.XPATH, xpath)
        results_text = [" ".join(result.text.split('\n')) for result in results_raw if result.text != '']
        return results_text

    def get_stops_from_wrappers(self, wrappers):
        departure_stops = ['' for _ in range(len(wrappers))]
        departure_n_stops = ['' for _ in range(len(wrappers))]
        return_stops = [0 for _ in range(len(wrappers))]
        return_n_stops = [0 for _ in range(len(wrappers))]

        base_index = 2

        for i, wrapper in enumerate(wrappers):
            departure_n_stops[i] = wrapper.split('\n')[base_index]
            if departure_n_stops[i] != 'nonstop': departure_stops[i] = wrapper.split('\n')[base_index+1]
            else: base_index -= 1

            return_n_stops[i] = wrapper.split('\n')[base_index+8]
            if return_n_stops[i] != 'nonstop': return_stops[i] = wrapper.split('\n')[base_index+9]
        
        return departure_n_stops, departure_stops, return_n_stops, return_stops

    def get_data_from_all_fields(self):
        departure_data = {}
        return_data = {}
        prices = []
        wrappers = []
        
        self.logger.log(f'Getting info from fields: {list(self.all_field_xpath)}')
        for key, val in self.all_field_xpath.items():
            # We dont actually want to process the wrapper so will call the raw method
            if key == 'Wrappers':
                wrappers = self.driver.find_elements(By.XPATH, self.all_field_xpath['Wrappers'])
                wrappers = [x.text for x in wrappers]
                continue

            results = self.get_data(val)
            
            # Price is the only field that's only one per result
            if key == 'Prices':
                prices = results
                continue
            
            departure_data[key] = results[::2]
            return_data[key] = results[1::2]
            
        return wrappers, departure_data, return_data, prices

    def _display_dic(self, dic):
        for key, val in dic.items():
            print(f'{len(val)}, {key}')

    def generate_flights_df(self, wrappers, departure_data, return_data, prices):
        self.logger.log('Creating data frames from fetched info')

        print(len(wrappers), len(prices))
        # Display dictionary lenghts
        print('departure')
        self._display_dic(departure_data)
        print('return')
        self._display_dic(return_data)

        # Creating both data frames for departure and return and putting them together
        try:
            departure_df = pd.DataFrame.from_dict(departure_data)
            return_df = pd.DataFrame.from_dict(return_data)
            full_df = pd.concat([departure_df, return_df], axis=1)
            self.logger.log('Successfully created dataframe...')
        except:
            self.logger.log('Dictionary dimensions are incorrect, trying to fix from raw wrapper data...')

            try:
                departure_n_stops, _, return_n_stops, _ = self.get_stops_from_wrappers(wrappers)
                departure_data['Stops'] = departure_n_stops
                return_data['Stops'] = return_n_stops

                departure_df = pd.DataFrame.from_dict(departure_data)
                return_df = pd.DataFrame.from_dict(return_data)
                full_df = pd.concat([departure_df, return_df], axis=1)
                self.logger.log('Successfully created dataframe...')
            except Exception as e:
                print(e)
                print('departure')
                self._display_dic(departure_data)
                print('return')
                self._display_dic(return_data)
        
        # Adding the multilevel column to the data frame
        column_array = [['Departure', 'Return'], departure_df.columns]
        column_index = pd.MultiIndex.from_product(column_array)
        full_df.columns = column_index
        
        # Adding the price column
        full_df['Price'] = prices
        
        return full_df
    
    def get_flights(self, origin, destination, departure_date, return_date, load_attemps=1, modes=['best', 'cheap', 'quick']):
        self.logger.log(f'STARTING getting flights from {origin} to {destination}, from {departure_date} to {return_date}')

        dfs = []
        full_wrappers = []

        # Getting results for passed modes
        for mode in modes:
            self.logger.log(f'TRYING info scrape for mode {mode}')
            # Getting the webpage
            self.start_kayak(origin, destination, departure_date, return_date, None)

            # Clicking the current mode button
            self.change_mode_with_button(mode)

            # Loading more results +15 results per load approx.
            for _ in range(load_attemps):
                self.load_more() 

            wrappers, departure_data, return_data, prices = self.get_data_from_all_fields()
            dfs.append(self.generate_flights_df(wrappers, departure_data, return_data, prices))
            full_wrappers += wrappers

        # Concatenating all dataframes into one
        full_df = pd.concat(dfs, ignore_index=True)
        return full_df, wrappers

In [31]:
scrapper = Flight_Scrapper()

flights, wrappers = scrapper.get_flights('CMH', 'ROM', '2023-07-19', '2023-07-23', load_attemps=3)

Initiating Chrome driver instance
Waiting to make sure driver starts properly, sleeping...
STARTING getting flights from CMH to ROM, from 2023-07-19 to 2023-07-23
TRYING info scrape for mode best
Trying to get https://www.kayak.com/flights/CMH-ROM/2023-07-19/2023-07-23
Successfully accessed kayak, sleeping...
Changing to best mode.
best mode active, sleeping...
Attempting to load more results...
More results loaded, sleeping...
Attempting to load more results...
More results loaded, sleeping...
Attempting to load more results...
More results loaded, sleeping...
Getting info from fields: ['Wrappers', 'Depart_Time', 'Arrival_Time', 'Stops', 'Duration', 'Cities', 'Prices']
Creating data frames from fetched info
62 62
departure
62, Depart_Time
62, Arrival_Time
62, Stops
62, Duration
62, Cities
return
62, Depart_Time
62, Arrival_Time
62, Stops
62, Duration
62, Cities
Successfully created dataframe...
TRYING info scrape for mode cheap
Trying to get https://www.kayak.com/flights/CMH-ROM/2023-

In [ ]:
flights.to_excel('My_output.xlsx', index='False')
flights.iloc[1]

In [32]:
def sort_by_price(flights):
    def extract_price(s):
            return int(s[1:].replace(',', ''))
        
    sorted_flights = flights.copy()

    sorted_flights['real_price'] = sorted_flights['Price'].apply(extract_price)
    sorted_flights.sort_values('real_price', inplace=True)
    sorted_flights.drop('real_price', axis=1, inplace=True)
    
    return sorted_flights

In [33]:
sorted_flights = sort_by_price(flights)

In [35]:
### lets print first 5 rows of data..
import pandas as pd
from pandas import option_context

with option_context("display.max_rows", None,"display.max_colwidth",60,"display.max_columns",50):
        display(sorted_flights.head(100))

Departure                                                             \
     Depart_Time     Arrival_Time                 Stops Duration     Cities   
62       2:32 pm       8:20 am +1            1 stop ORD  11h 48m  CMH ‐ FCO   
79       7:00 am      10:55 am +1            1 stop EWR  21h 55m  CMH ‐ FCO   
78       7:00 am      10:55 am +1            1 stop EWR  21h 55m  CMH ‐ FCO   
77       8:17 am      10:55 am +1            1 stop EWR  20h 38m  CMH ‐ FCO   
76       6:00 am       8:20 am +1            1 stop ORD  20h 20m  CMH ‐ FCO   
74       8:17 am      10:55 am +1            1 stop EWR  20h 38m  CMH ‐ FCO   
73       6:04 am       7:45 am +1            1 stop EWR  19h 41m  CMH ‐ FCO   
72       6:00 am       8:20 am +1            1 stop ORD  20h 20m  CMH ‐ FCO   
71       7:35 am       9:35 am +1      2 stops ORD, YUL  20h 00m  CMH ‐ FCO   
75       7:35 am       9:35 am +1      2 stops ORD, YUL  20h 00m  CMH ‐ FCO   
69       7:35 am       8:20 am +1            1 stop ORD  18h 45m  CMH ‐ FCO   
70       6:04 am       7:45 am +1            1 stop EWR  19h 41m  CMH ‐ FCO   
64      11:05 am       8:20 am +1            1 stop ORD  15h 15m  CMH ‐ FCO   
2        2:32 pm       8:20 am +1            1 stop ORD  11h 48m  CMH ‐ FCO   
65      10:20 am       8:15 am +1            1 stop IAD  15h 55m  CMH ‐ FCO   
63       2:32 pm       8:20 am +1            1 stop ORD  11h 48m  CMH ‐ FCO   
67      10:20 am       8:15 am +1            1 stop IAD  15h 55m  CMH ‐ FCO   
68       7:35 am       8:20 am +1            1 stop ORD  18h 45m  CMH ‐ FCO   
66      11:05 am       8:20 am +1            1 stop ORD  15h 15m  CMH ‐ FCO   
80       7:23 pm      12:55 pm +1            1 stop JFK  11h 32m  CMH ‐ FCO   
87      10:20 am      10:55 am +1  2 stops IAD-DCA, EWR  18h 35m  CMH ‐ FCO   
81      10:20 am       8:20 am +1      2 stops IAD, ORD  16h 00m  CMH ‐ FCO   
82      10:20 am       8:20 am +1      2 stops IAD, ORD  16h 00m  CMH ‐ FCO   
83      11:05 am      10:55 am +1  2 stops ORD, LGA-EWR  17h 50m  CMH ‐ FCO   
84       8:17 am       8:15 am +1  2 stops EWR, DCA-IAD  17h 58m  CMH ‐ FCO   
85       8:17 am       8:15 am +1  2 stops EWR-LGA, IAD  17h 58m  CMH ‐ FCO   
86      10:20 am      10:55 am +1  2 stops IAD-DCA, EWR  18h 35m  CMH ‐ FCO   
90      11:05 am      10:55 am +1  2 stops ORD, LGA-EWR  17h 50m  CMH ‐ FCO   
91       8:17 am       8:15 am +1  2 stops EWR, DCA-IAD  17h 58m  CMH ‐ FCO   
89      10:20 am      10:55 am +1      2 stops IAD, EWR  18h 35m  CMH ‐ FCO   
98       7:00 am       8:15 am +1  2 stops EWR, DCA-IAD  19h 15m  CMH ‐ FCO   
97      10:20 am      10:55 am +1  2 stops IAD-DCA, EWR  18h 35m  CMH ‐ FCO   
96      10:20 am      10:55 am +1  2 stops IAD-DCA, EWR  18h 35m  CMH ‐ FCO   
88      10:20 am      10:55 am +1  2 stops IAD-DCA, EWR  18h 35m  CMH ‐ FCO   
94      10:20 am      10:55 am +1      2 stops IAD, EWR  18h 35m  CMH ‐ FCO   
95      10:20 am      10:55 am +1  2 stops IAD-DCA, EWR  18h 35m  CMH ‐ FCO   
93       7:00 am       8:15 am +1  2 stops EWR, DCA-IAD  19h 15m  CMH ‐ FCO   
92       8:17 am       8:15 am +1  2 stops EWR-LGA, IAD  17h 58m  CMH ‐ FCO   
103     11:34 am      12:55 pm +1      2 stops EWR, YYZ  19h 21m  CMH ‐ FCO   
102     11:34 am      12:55 pm +1      2 stops EWR, YYZ  19h 21m  CMH ‐ FCO   
100      1:35 pm      12:55 pm +1      2 stops EWR, YYZ  17h 20m  CMH ‐ FCO   
99       7:23 pm      12:55 pm +1            1 stop JFK  11h 32m  CMH ‐ FCO   
21       7:23 pm      12:55 pm +1            1 stop JFK  11h 32m  CMH ‐ FCO   
101      1:35 pm      12:55 pm +1      2 stops EWR, YYZ  17h 20m  CMH ‐ FCO   
104      7:23 pm      12:55 pm +1            1 stop JFK  11h 32m  CMH ‐ FCO   
106      6:04 am      10:55 am +1            1 stop EWR  22h 51m  CMH ‐ FCO   
107      6:04 am      10:55 am +1            1 stop EWR  22h 51m  CMH ‐ FCO   
108      7:23 pm      10:10 am +2      2 stops JFK, HEL  32h 47m  CMH ‐ FCO   
105      7:23 pm       6:45 pm +1      2 stops JFK, HEL  